In [5]:
import sc2reader
from sc2reader.engine.plugins import APMTracker, ContextLoader, SelectionTracker
from sc2reader import events, data
import pandas as pd
import glob
from IPython.display import display
import json
from collections import Counter

pd.options.display.max_columns = None

In [6]:
with open('./stats.json', 'rb') as f:
    stats_attrs = json.load(f)

unit_data = json.loads(data.unit_data)

unit_map = {}
for k in unit_data:
    unit_map[k] = list(unit_data[k].keys())

In [7]:
replays = []
paths = [path for path in glob.glob('./_data/**/*.SC2Replay', recursive=True)]

count = 999999

count = count if count <= len(paths) else len(paths)

for i, path in enumerate(paths[:count]):
    print('\rLoading replays... %6.2f%% complete!' % ((i+1)/count*100), end='', flush=True)
    
    replay = sc2reader.load_replay(path, engine=sc2reader.engine.GameEngine(plugins=[ContextLoader(), APMTracker(), SelectionTracker()]))
    replays.append(replay)

100.00% complete

In [8]:
for event in replays[0].messages:
    print('%s\tmessage="%s"' % (event, event.text))

replays[0].players[0].name
print(replays[0].players[0], replays[0].players[0].result)

00.00	Seither         ChatEvent	message="glhf"
00.10	Probe           ChatEvent	message="gl hf"
14.28	Seither         ChatEvent	message="gg"
Player 1 - Seither (Terran) Loss


In [9]:
df_data = []

valid_games = 0

pt_dict = dict.fromkeys(unit_map['Protoss'] + unit_map['Terran'], 0)

for i, replay in enumerate(replays[:]):

	if len(set([replay.players[0].pick_race[0], replay.players[1].pick_race[0]]) & set(['P','T'])) == 2:
		print('\n{} Game #{:03} | {} vs. {} {}'.format('-'*17, i+1, replay.players[0].pick_race, replay.players[1].pick_race, '-'*17))

		valid_games += 1
		dd = {}

		for event in replay.events:
			if isinstance(event, events.PlayerLeaveEvent):
				print('Player {} left {} seconds into the game.'.format(event.player, event.second))
				break

			if event.second % 30 == 0:

				# every 10 seconds
				if isinstance(event, events.PlayerStatsEvent):
					d = {}

					is_player_1 = replay.players[1].pid == event.pid
					race = replay.players[is_player_1].pick_race[0]
					win = replay.players[is_player_1].result == 'Win'

					lower_bound = 0 if event.second == 0 else event.second-30
					ap30s = sum(list(replay.players[is_player_1].aps.values())[lower_bound:event.second])

					d['match_id'] = i
					d['frame'] = event.frame
					d['second'] = event.second
					d['race'] = race
					d['ap30s'] = ap30s

					for attr in stats_attrs['PlayerStatsEvent']:
						d[attr] = eval('event.' + attr)
					
					d['win'] = win

					dd[race] = d

				# every 15 seconds
				if isinstance(event, events.UnitPositionsEvent):
					dd1 = dd['P']
					dd2 = dd['T']

					dd1.update(pt_dict)
					dd2.update(pt_dict)

					current_units = [str(a).split(' ')[0].lower() for a in event.units.keys()]
					counted_units = Counter(current_units)

					for k in counted_units:
						if k in unit_map['Protoss']:
							dd1[k] = counted_units[k]
						elif k in unit_map['Terran']:
							dd2[k] = counted_units[k]
						else:
							print('Found invalid unit "{}".'.format(k))

					df_data.extend([dd1, dd2])
					dd = {}

df = pd.DataFrame(df_data)
print('\nEND: ({}, {}) found {} valid games out of {}.'.format(*df.shape, valid_games, len(replays)))

otoss) left 1324 seconds into the game.

----------------- Game #271 | Terran vs. Protoss -----------------
Found invalid unit "liberatorag".
Found invalid unit "liberatorag".
Found invalid unit "liberator".
Found invalid unit "liberator".
Player Player 2 - JieShi (Protoss) left 1269 seconds into the game.

----------------- Game #272 | Protoss vs. Terran -----------------
Player Player 1 - JieShi (Protoss) left 510 seconds into the game.

----------------- Game #285 | Protoss vs. Terran -----------------
Player Player 2 - TIME (Terran) left 849 seconds into the game.

----------------- Game #286 | Protoss vs. Terran -----------------
Player Player 1 - iGMacSed (Protoss) left 488 seconds into the game.

----------------- Game #287 | Terran vs. Protoss -----------------
Player Player 2 - iGMacSed (Protoss) left 1071 seconds into the game.

----------------- Game #288 | Protoss vs. Terran -----------------
Player Player 1 - iGMacSed (Protoss) left 758 seconds into the game.

------------

In [10]:
df.describe()

,match_id,frame,second,ap30s,minerals_current,vespene_current,minerals_collection_rate,vespene_collection_rate,workers_active_count,minerals_used_in_progress,vespene_used_in_progress,minerals_used_current,vespene_used_current,minerals_lost,vespene_lost,minerals_killed,vespene_killed,food_used,food_made,minerals_used_active_forces,vespene_used_active_forces,adept,archon,assimilator,carrier,colossus,cyberneticscore,darkshrine,darktemplar,disruptor,fleetbeacon,forge,gateway,hightemplar,immortal,interceptor,mothership,mothershipcore,nexus,observer,oracle,phoenix,photoncannon,probe,pylon,reactor,roboticsbay,roboticsfacility,sentry,stalker,stargate,tempest,templararchive,twilightcouncil,voidray,warpgate,warpprism,warpprismphasing,zealot,armory,autoturret,banshee,barracks,barrackstechlab,barracksreactor,barracksflying,battlecruiser,battlehellion,bunker,commandcenter,commandcenterflying,cyclone,engineeringbay,factory,factoryflying,factoryreactor,factorytechlab,fusioncore,ghost,ghostacademy,hellion,marauder,marine,medivac,missileturret,mule,orbitalcommand,orbitalcommandflying,planetaryfortress,raven,reaper,refinery,scv,sensortower,siegetank,siegetanksieged,starport,starportflying,starportreactor,starporttechlab,supplydepot,supplydepotlowered,techlab,thor,viking,vikingassault,warhound,widowmine,widowmineburrowed
count,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.0,6490.000000,6490.000000,6490.0,6490.0,6490.000000,6490.00000,6490.0,6490.0,6490.0,6490.000000,6490.000000,6490.000000,6490.000000,6490.0,6490.0,6490.000000,6490.000000,6490.000000,6490.0,6490.000000,6490.0,6490.0,6490.0,6490.0,6490.000000,6490.000000,6490.0,6490.000000,6490.0,6490.0,6490.000000,6490.0,6490.000000,6490.000000,6490.000000,6490.0,6490.0,6490.000000,6490.000000,6490.0,6490.0,6490.000000,6490.000000,6490.000000,6490.0,6490.000000,6490.000000,6490.000000,6490.0,6490.000000,6490.000000,6490.0,6490.0,6490.0,6490.000000,6490.0,6490.000000,6490.000000,6490.000000,6490.000000,6490.0,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.000000,6490.0,6490.000000,6490.0,6490.000000,6490.000000,6490.000000,6490.000000,6490.0,6490.0,6490.0,6490.0,6490.0,6490.000000,6490.000000,6490.000000,6490.0,6490.000000,6490.000000
mean,410.528814,11617.183359,726.073960,142.705948,380.175347,356.825578,1852.098305,598.315100,48.117411,669.094761,195.057781,9155.040832,1563.436055,3982.955932,1190.216795,4031.607088,1188.567026,99.094761,123.712018,2328.365177,1106.556240,0.075193,0.072111,0.0,0.005547,0.120955,0.0,0.0,0.061017,0.05470,0.0,0.0,0.0,0.027273,0.058398,0.091371,0.000154,0.0,0.0,0.004160,0.018798,0.153621,0.0,0.190909,0.0,0.0,0.0,0.0,0.093837,1.015562,0.0,0.009245,0.0,0.0,0.018798,0.0,0.013251,0.005239,0.493220,0.0,0.0,0.009245,0.002619,0.0,0.0,0.001541,0.005855,0.010169,0.0,0.000308,0.001233,0.045300,0.0,0.001849,0.000462,0.0,0.0,0.0,0.126810,0.0,0.048382,0.998613,1.710940,0.104777,0.0,0.002465,0.006471,0.004006,0.004468,0.008629,0.048228,0.0,0.227581,0.0,0.024499,0.097689,0.001849,0.000770,0.0,0.0,0.0,0.0,0.0,0.002928,0.093220,0.035131,0.0,0.008629,0.042373
std,253.307329,6838.222895,427.388931,52.308677,566.133577,509.030928,726.527227,341.132233,19.146194,385.843928,190.909973,5273.263487,1378.685027,6013.438540,2055.509564,6080.545354,2050.610010,54.896777,70.607252,1921.304692,1053.827153,0.349207,0.394962,0.0,0.144664,0.485266,0.0,0.0,0.558193,0.33577,0.0,0.0,0.0,0.258668,0.316707,1.547263,0.012413,0.0,0.0,0.066722,0.163614,0.861484,0.0,0.795315,0.0,0.0,0.0,0.0,0.431795,2.388052,0.0,0.205280,0.0,0.0,0.229476,0.0,0.114357,0.072195,1.649341,0.0,0.0,0.112033,0.051117,0.0,0.0,0.078498,0.139224,0.246201,0.0,0.017553,0.035090,0.358651,0.0,0.042964,0.021497,0.0,0.0,0.0,0.611272,0.0,0.337425,2.747868,3.919197,0.475109,0.0,0.063251,0.0

In [11]:
df.to_csv('./_sc2_{}_{}{}.csv'.format(valid_games, 'P', 'T'))